In [0]:
%python
spark.sql("GRANT SELECT ON TABLE silver_dataprocessing.default.silver_agoda_reviews_details_2 TO `23104111@siswa365.um.edu.my`")

DataFrame[]

In [0]:
spark.sql("GRANT SELECT ON TABLE silver_dataprocessing.default.silver_agoda_reviews_details_2 TO `eia190531@siswa365.um.edu.my`")

DataFrame[]

In [0]:
spark.sql("GRANT SELECT ON SCHEMA silver_dataprocessing.default TO `eia190531@siswa365.um.edu.my`")

DataFrame[]

In [0]:
# Train Multiple Models with Hyperparameter
import pandas as pd
import mlflow
import mlflow.sklearn
import io
import joblib
import os

from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# Load and convert data
df = spark.sql("SELECT content_no_emojis, sentiment FROM silver_dataprocessing.default.silver_agoda_reviews_details_2")
pdf = df.toPandas()

# TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(pdf["content_no_emojis"].astype(str))

# Encode labels
label_map = {"Positive": 1, "Neutral": 0, "Negative": 2}
y = pdf["sentiment"].map(label_map)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Models
models = {
    "Logistic Regression": LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=150, max_depth=15, min_samples_split=5, random_state=42),
    "Naive Bayes": MultinomialNB(alpha=0.5),
    "Linear SVM": LinearSVC(C=0.8, max_iter=2000)
}

best_accuracy = 0
best_model = None
best_model_name = ""
best_report = ""

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)

    print(f"=== {name} ===")
    print("Accuracy:", acc)
    print(report)
    print("-" * 50)

    if acc > best_accuracy:
        best_accuracy = acc
        best_model_name = name
        best_model = model
        best_report = report

print(f"✅ Best model is '{best_model_name}' with accuracy {best_accuracy:.4f}")


# Save best_model to binary in memory
model_buffer = io.BytesIO()
joblib.dump(best_model, model_buffer)
model_bytes = model_buffer.getvalue()

# Save vectorizer to binary in memory
vectorizer_buffer = io.BytesIO()
joblib.dump(vectorizer, vectorizer_buffer)
vectorizer_bytes = vectorizer_buffer.getvalue()

# Convert to DataFrame and save to Delta as binary
spark = SparkSession.builder.getOrCreate()

model_df = spark.createDataFrame([(model_bytes,)], ["model_bin"])
model_df.write.mode("overwrite").save("dbfs:/models/best_model_bin")

vectorizer_df = spark.createDataFrame([(vectorizer_bytes,)], ["vectorizer_bin"])
vectorizer_df.write.mode("overwrite").save("dbfs:/models/tfidf_vectorizer_bin")

print("✅ Model and vectorizer saved to dbfs:/models/")

# Log to MLflow
try:
    with mlflow.start_run():
        mlflow.sklearn.log_model(best_model, "best_model")
        mlflow.log_param("model_name", best_model_name)
        mlflow.log_metric("best_accuracy", best_accuracy)
        print("✅ Model logged to MLflow (artifact only)")
except Exception as e:
    print(f"⚠️ MLflow logging skipped due to error: {e}")

# Save evaluation report to Delta table
report_data = [(best_model_name, float(best_accuracy), best_report)]
report_schema = ["model_name", "accuracy", "classification_report"]
spark_report_df = spark.createDataFrame(report_data, report_schema)

spark_report_df.write.mode("overwrite").format("delta").saveAsTable("default.gold_model_results")
print("✅ Best model report saved to Delta table: default.gold_model_results")

# Export CSV for Power BI
try:
    spark_report_df.toPandas().to_csv("/dbfs/FileStore/silver_dataprocessing.default.gold_model_results.csv", index=False)
    print("✅ Predictions saved to CSV for Power BI.")
except Exception as e:
    print(f"⚠️ Failed to export CSV: {e}")


In [0]:
# ✅ Create a Spark DataFrame with the best model evaluation
report_data = [(best_model_name, float(best_accuracy), best_report)]
report_schema = ["model_name", "accuracy", "classification_report"]

spark_report_df = spark.createDataFrame(report_data, report_schema)

# 💾 Save the report as a Delta table with consistent name
spark_report_df.write.mode("overwrite").format("delta").saveAsTable("default.gold_model_results")
print("✅ Best model report saved to Delta table: default.gold_model_results")


✅ Best model report saved to Delta table: default.gold_model_results


In [0]:
# ✅ Save predictions to a CSV file in DBFS for Power BI
csv_temp_path = "/tmp/gold_model_results.csv"

try:
    spark_report_df.toPandas().to_csv(csv_temp_path, index=False)
    print(f"✅ Predictions saved temporarily to: {csv_temp_path}")
except Exception as e:
    print(f"⚠️ Failed to export CSV to /tmp: {e}")



✅ Predictions saved temporarily to: /tmp/gold_model_results.csv


In [0]:
%sql
SELECT * FROM default.gold_model_results;


model_name,accuracy,classification_report
Linear SVM,0.8709960508995174,precision recall f1-score support 0 0.79 0.83 0.81 368 1 0.93 0.89 0.91 1307 2 0.80 0.85 0.83 604 accuracy 0.87 2279 macro avg 0.84 0.86 0.85 2279 weighted avg 0.87 0.87 0.87 2279


In [0]:
%python
# Temporarily test loading latest data instead of yesterday
df = (
    spark.sql("""
        SELECT content_no_emojis, score, repliedAt
        FROM silver_dataprocessing.default.silver_agoda_reviews_details
    """)
    .orderBy(col("repliedAt").desc())
    .limit(1000)
)

display(df)

content_no_emojis,score,repliedAt
"Agoda is a popular hotel booking app known for its wide selection of accommodations worldwide and competitive prices. Whether you’re planning a luxury getaway or a budget trip, Agoda offers great deals with a user-friendly interface that makes booking simple and fast.",5,2025-06-03T06:37:34
DO NOT USE THIS APP OR BOOK WITH AGODA!!! I booked a room and canceled 3 days before and they still charged me even though it says anything before 24hrs can be canceled. I contacted the hotel manager to dispute it and they said they have no record of my booking so agoda stole my money. and they change the prices on you for the hotel listing.,1,2025-06-03T05:03:49
ลงแล้วทำเครื่องค้างๆ เปิดแอพอื่นแล้ว crashed เปิดไม่ได้,1,2025-06-03T04:59:45
PayNow QR Payment does not work. Generated QR code cannot be detected,1,2025-06-03T03:59:07
feedback,5,2025-06-03T00:27:34


In [0]:
from pyspark.sql.functions import to_date

# Show the most recent repliedAt dates available in your data
df_debug = spark.sql("""
    SELECT DISTINCT to_date(repliedAt) AS replied_date
    FROM silver_dataprocessing.default.silver_agoda_reviews_details
    ORDER BY replied_date DESC
""")

df_debug.show(10)


+------------+
|replied_date|
+------------+
|  2025-06-03|
+------------+



In [0]:
# ✅ Daily Inference Script: Load new data, predict sentiment using pre-trained model, and save for reporting

import pandas as pd
import joblib
import io
from datetime import datetime, timedelta
from pyspark.sql.functions import col, to_date, lit
from pyspark.sql.types import DateType

# Define yesterday's Malaysia date (UTC+8)
yesterday_local = datetime.strptime("2025-06-03", "%Y-%m-%d").date()  # for manual testing

df = (
    spark.sql("""
        SELECT content_no_emojis, score, repliedAt
        FROM silver_dataprocessing.default.silver_agoda_reviews_details
    """)
    .withColumn("replied_date", to_date(col("repliedAt")))
    .filter(col("replied_date") == lit(yesterday_local).cast(DateType()))
)

row_count = df.count()
print(f"📦 Loaded {row_count} rows for Malaysia date: {yesterday_local}")

if row_count == 0:
    print("⚠️ No new data available for prediction.")
else:
    # 🧪 Convert to Pandas
    pdf = df.select("content_no_emojis", "score", "repliedAt").toPandas()

    # ✅ Load model and vectorizer from DBFS binary blobs
    try:
        model_bytes = spark.read.load("dbfs:/models/best_model_bin").collect()[0]["model_bin"]
        vectorizer_bytes = spark.read.load("dbfs:/models/tfidf_vectorizer_bin").collect()[0]["vectorizer_bin"]

        model = joblib.load(io.BytesIO(model_bytes))
        vectorizer = joblib.load(io.BytesIO(vectorizer_bytes))

        print("✅ Pretrained model and vectorizer loaded from DBFS.")
    except Exception as e:
        print(f"❌ Failed to load model/vectorizer from DBFS: {e}")
        raise

    # 🧼 Vectorize input
    X = vectorizer.transform(pdf["content_no_emojis"].astype(str))

    # 🔮 Predict sentiment
    label_map_reverse = {1: "Positive", 0: "Neutral", 2: "Negative"}
    y_pred = model.predict(X)
    pdf["predicted_sentiment"] = pd.Series(y_pred).map(label_map_reverse)

    # 💾 Convert to Spark DataFrame
    result_df = spark.createDataFrame(pdf)

    # Save to DBFS as Spark DataFrame CSV 
    output_path = "dbfs:/FileStore/predicted_sentiments_yesterday.csv"

try:
    result_df.write.mode("overwrite").option("header", "true").csv(output_path)
    print(f"✅ Inference report saved as CSV to: {output_path}")
except Exception as e:
    print(f"⚠️ Failed to save CSV to DBFS: {e}")


📦 Loaded 5 rows for Malaysia date: 2025-06-03
✅ Pretrained model and vectorizer loaded from DBFS.
✅ Inference report saved as CSV to: dbfs:/FileStore/predicted_sentiments_yesterday.csv


In [0]:
# ✅ Load predicted sentiment CSV from DBFS
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("dbfs:/FileStore/predicted_sentiments_yesterday.csv")
)

# Show a few rows
df.show(truncate=False)

# Count the number of rows
print(f"✅ Loaded {df.count()} rows from predicted_sentiments_yesterday.csv")


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------------------+-------------------+
|content_no_emojis                                                                                                                                                                                                                                                                                                                                      |score|repliedAt          |predicted_sentiment|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------